# Global Terrorism Attacks - Predicting The Responsible Group

**Table of Contents**
<div id="toc"></div>

## Load data

In [18]:
%matplotlib inline

import pandas as pd
import csv
from sklearn import preprocessing
import os.path

gtdConvertedFilename = 'gtd_converted.csv'

if not os.path.isfile(gtdConvertedFilename):
    %run CreateCSVFile.py

gtd = pd.read_csv(gtdConvertedFilename, encoding='latin1', low_memory=False)
gtd.tail(3)

,Unnamed: 0,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
170347,170347,201612310043,2016,12,31,NaN,1,NaN,229,Democratic Republic of the Congo,...,NaN,"""DRC Armed Forces Dislodge Ugandan LRA Rebels ...",NaN,NaN,START Primary Collection,1,1,0,1,NaN
170348,170348,201612310044,2016,12,31,NaN,0,NaN,130,Mexico,...,There is doubt that this incident meets terror...,"""Gunmen shoot at home of Mexican indigenous ri...","""â Gunmen shoot at home of Mexican indigenou...",NaN,START Primary Collection,-9,-9,0,-9,NaN
170349,170349,201701270001,2016,12,30,NaN,0,NaN,195,Sudan,...,There is doubt that this incident meets terror...,"""Rebels To Blame For Nierteti Killing: Central...","""Security deteriorates in West Darfur alongsid...",NaN,START Primary Collection,0,0,0,0,NaN


## Preprocessing

### Filter Groups

In [4]:
from collections import Counter

group_count = Counter(gtd['gname'])
# Remove groups with 3 attacks or less
filtered_groups = [group for group, counter in group_count.items() if counter > 3]
# Remove 'Unknown'
filtered_groups.remove('Unknown')

gtd = gtd[gtd['gname'].isin(filtered_groups)]

gtd.shape

(55568, 136)

### Define Features

In [5]:
#columns_to_keep = ['gname', 'iyear', 'country', 'region', 'crit1', 'crit2', 'crit3', 'attacktype1', 'targtype1', 'targsubtype1','natlty1', 'weaptype1','weapsubtype1', 'nperps', 'nkill', 'nwound', 'ransom']
columns_to_keep = ['gname', 'iyear', 'country', 'crit1', 'crit2', 'crit3', 'attacktype1', 'targtype1', 'targsubtype1',
'weaptype1', 'weapsubtype1', 'ransom']
# columns_to_keep = ['gname', 'iyear', 'country', 'region', 'attacktype1', 'weaptype1']
gtd = gtd[columns_to_keep]
gtd.tail(3)

,gname,iyear,country,crit1,crit2,crit3,attacktype1,targtype1,targsubtype1,weaptype1,weapsubtype1,ransom
99964,Jharkhand Liberation Tigers (JLT),2011,92,1,0,1,6,17,NaN,9.0,23.0,-9.0
99965,Tehrik-i-Taliban Pakistan (TTP),2011,153,1,1,1,9,14,67.0,13.0,NaN,NaN
99966,Al-Shabaab,2011,182,1,1,1,2,14,67.0,5.0,5.0,NaN


### Transform Target

In [6]:
number = preprocessing.LabelEncoder()
gtd['gname'] = number.fit_transform(gtd.gname) #number.inverse_transform()
gtd = gtd.rename(columns={'gname': 'groupId'})

### Fill NaNs

In [7]:
gtd=gtd.fillna(0) #TODO: 999 or 0?

gtd.tail(3)

,groupId,iyear,country,crit1,crit2,crit3,attacktype1,targtype1,targsubtype1,weaptype1,weapsubtype1,ransom
99964,379,2011,92,1,0,1,6,17,0.0,9.0,23.0,-9.0
99965,758,2011,153,1,1,1,9,14,67.0,13.0,0.0,0.0
99966,43,2011,182,1,1,1,2,14,67.0,5.0,5.0,0.0


In [8]:
gtd.to_csv('gtd_processed_11features.csv', encoding='utf-8', index=False)